In [4]:
import os
import random
import shutil

def split_dataset(image_dir, label_dir, train_ratio=0.8):
    # Ensure paths are correct
    image_dir = os.path.abspath(image_dir)
    label_dir = os.path.abspath(label_dir)
    
    # Debugging: Print paths to ensure they are correct
    print(f"Image directory: {image_dir}")
    print(f"Label directory: {label_dir}")

    # List all images and labels
    try:
        images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
        labels = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    except OSError as e:
        print(f"Error accessing directories: {e}")
        return

    # Ensure there is a corresponding label file for each image
    images = [img for img in images if os.path.exists(os.path.join(label_dir, img.replace('.jpg', '.txt')))]
    labels = [lbl for lbl in labels if os.path.exists(os.path.join(image_dir, lbl.replace('.txt', '.jpg')))]

    # Debugging: Print counts of images and labels
    print(f"Total images: {len(images)}")
    print(f"Total labels: {len(labels)}")

    # Shuffle the dataset
    combined = list(zip(images, labels))
    random.shuffle(combined)
    images[:], labels[:] = zip(*combined)

    # Split the dataset
    split_index = int(train_ratio * len(images))
    train_images, val_images = images[:split_index], images[split_index:]
    train_labels, val_labels = labels[:split_index], labels[split_index:]

    # Create directories for the splits
    for split in ['train', 'val']:
        os.makedirs(os.path.join(image_dir, split), exist_ok=True)
        os.makedirs(os.path.join(label_dir, split), exist_ok=True)

    # Move the files to their respective directories
    for img in train_images:
        shutil.move(os.path.join(image_dir, img), os.path.join(image_dir, 'train', img))
    for lbl in train_labels:
        shutil.move(os.path.join(label_dir, lbl), os.path.join(label_dir, 'train', lbl))
    for img in val_images:
        shutil.move(os.path.join(image_dir, img), os.path.join(image_dir, 'val', img))
    for lbl in val_labels:
        shutil.move(os.path.join(label_dir, lbl), os.path.join(label_dir, 'val', lbl))

    # Debugging: Print final counts of images and labels in each set
    print(f"Training set: {len(train_images)} images, {len(train_labels)} labels")
    print(f"Validation set: {len(val_images)} images, {len(val_labels)} labels")

def create_yaml_file(dataset_dir):
    yaml_content = f"""
train: {os.path.join(dataset_dir, 'images/train')}
val: {os.path.join(dataset_dir, 'images/val')}

nc: 4
names: ['tank', 'building', 'tree', 'human']
"""
    with open(os.path.join(dataset_dir, 'dataset.yaml'), 'w') as file:
        file.write(yaml_content)

# Path to your dataset
dataset_dir = r'D:\tank_rec'
image_dir = os.path.join(dataset_dir, 'images')
label_dir = os.path.join(dataset_dir, 'labels')

# Split the dataset
split_dataset(image_dir, label_dir, train_ratio=0.8)

# Create or update the dataset.yaml file
create_yaml_file(dataset_dir)


Image directory: D:\tank_rec\images
Label directory: D:\tank_rec\labels
Total images: 780
Total labels: 780
Training set: 624 images, 624 labels
Validation set: 156 images, 156 labels


In [1]:
import torch
from ultralytics import YOLO

# Check PyTorch installation
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Load YOLOv8 model
model = YOLO('yolov8m.pt')  # You can choose a different model size if needed
print("YOLOv8 model loaded successfully")


PyTorch version: 2.3.1
CUDA available: True
YOLOv8 model loaded successfully


In [2]:
import os
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8m.pt')  # You can choose a different model size if needed

# Path to the dataset YAML file
dataset_yaml = os.path.join(r'D:\tank_rec', 'dataset.yaml')

# Train the model
model.train(data=dataset_yaml, epochs=100, imgsz=640, batch=16)

# Evaluate the model
results = model.val(data=dataset_yaml)




Ultralytics YOLOv8.2.38  Python-3.9.19 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=D:\tank_rec\dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show

train: Scanning D:\tank_rec\labels\train.cache... 624 images, 0 backgrounds, 0 corrupt: 100%|██████████| 624/624 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [5]:
import os
import torch
from ultralytics import YOLO

def train_yolo_model():
    try:
        # Clear GPU cache
        torch.cuda.empty_cache()

        # Load the YOLOv8 model
        print("Loading YOLOv8 model...")
        model = YOLO('yolov8m.pt')  # You can choose a different model size if needed

        # Path to the dataset YAML file
        dataset_yaml = os.path.join(r'D:\tank_rec', 'dataset.yaml')

        # Train the model
        print("Starting training...")
        model.train(data=dataset_yaml, epochs=100, imgsz=512, batch=4, workers=4, mosaic=0)

        # Evaluate the model
        print("Evaluating the model...")
        results = model.val(data=dataset_yaml)
        print("Evaluation results:", results)

    except RuntimeError as e:
        print(f"RuntimeError: {e}")
        if "out of memory" in str(e):
            print("CUDA out of memory. Try reducing batch size or image size.")
        else:
            print("Unexpected error occurred during training or evaluation.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        print("Training and evaluation process completed.")

if __name__ == "__main__":
    train_yolo_model()



Loading YOLOv8 model...
Starting training...
Ultralytics YOLOv8.2.38  Python-3.9.19 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=D:\tank_rec\dataset.yaml, epochs=100, time=None, patience=100, batch=4, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=4, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fa

train: Scanning D:\tank_rec\labels\train.cache... 624 images, 0 backgrounds, 0 corrupt: 100%|██████████| 624/624 [00:00<?, ?it/s]
val: Scanning D:\tank_rec\labels\val.cache... 156 images, 0 backgrounds, 0 corrupt: 100%|██████████| 156/156 [00:00<?, ?it/s]


Plotting labels to runs\detect\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 512 train, 512 val
Using 4 dataloader workers
Logging results to runs\detect\train7
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.05G      1.726      2.494      1.907          9        512: 100%|██████████| 156/156 [00:13<00:00, 11.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:01<00:00, 12.90it/s]

                   all        156        308      0.461      0.234      0.166     0.0626


...


100 epochs completed in 0.424 hours.
Optimizer stripped from runs\detect\train7\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\train7\weights\best.pt, 52.0MB

Validating runs\detect\train7\weights\best.pt...
Ultralytics YOLOv8.2.38  Python-3.9.19 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25842076 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 20/20 [00:01<00:00, 15.89it/s]


                   all        156        308      0.598      0.453      0.482      0.251
                  tank        156        177      0.857      0.864      0.895      0.562
              building         10         10      0.464        0.2      0.239     0.0898
                  tree         44         66      0.549      0.409       0.39      0.166
                 human         37         55      0.522      0.338      0.403      0.185
Speed: 0.1ms preprocess, 4.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train7
Evaluating the model...
Ultralytics YOLOv8.2.38  Python-3.9.19 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25842076 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning D:\tank_rec\labels\val.cache... 156 images, 0 backgrounds, 0 corrupt: 100%|██████████| 156/156 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:06<00:00,  6.23it/s]


                   all        156        308      0.529      0.427      0.451      0.248
                  tank        156        177      0.831      0.859      0.893      0.565
              building         10         10      0.303        0.1      0.137     0.0795
                  tree         44         66      0.534      0.439      0.398      0.163
                 human         37         55      0.448       0.31      0.377      0.185
Speed: 0.2ms preprocess, 10.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train72
Evaluation results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002436D66B520>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,  